# Similarity metrics e IR con Word Embeddings

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity

In [3]:
from nltk import sent_tokenize
from nltk.corpus import stopwords
import string
from tqdm import tqdm
import re

In [4]:
string.punctuation += '“”«»¿¡‘’'
string.punctuation = string.punctuation.replace('-', '')
table = str.maketrans({key: None for key in string.punctuation})
sw_list = stopwords.words('spanish')
print(string.punctuation)

!"#$%&'()*+,./:;<=>?@[\]^_`{|}~“”«»¿¡‘’


## Define functions

In [19]:
def preprocess(sent):
    return [token for token in 
            re.sub("\n+", " ", str(sent).lower().translate(table)).split(" ") 
            if token not in sw_list]

In [5]:
def generate_search_corpus(df, media_filt):
    wmd_corpus = []
    for idx, (title, text, media) in tqdm(enumerate(zip(df['title'],
                                                        df['text'],
                                                        df['media']))):
        if media == media_filt:
            article_proc = preprocess(title)
            try:
                for sent in sent_tokenize(text):
                    article_proc += preprocess(sent)
            except:
                pass
            wmd_corpus.append(article_proc)
        else:
            continue
    return wmd_corpus

## Data Loading

In [6]:
%%time
df = pd.read_excel('../../datasets/spanish_news_corpus.xlsx')

CPU times: user 97.8 ms, sys: 4.91 s, total: 5.01 s
Wall time: 5.69 s


In [7]:
print(df.shape)
df.tail()

(39352, 6)


,date,title,text,keywords,media,url
39347,2019-10-23,Campofrío esquiva los aranceles de EEUU y la c...,La segunda mayor empresa de alimentación españ...,facturación|proteínas|esquiva|porcina|arancele...,expansion,https://www.expansion.com/empresas/distribucio...
39348,2019-10-23,"Capital Group aflora el 3,1% del capital de Ce...",La gestora de fondos estadounidense Capital Gr...,management|capital|través|euros|principal|comp...,expansion,https://www.expansion.com/mercados/2019/10/23/...
39349,2019-10-23,Se equivoca al rellenar la lotería y gana dos ...,Las personas que creen en el destino suelen pe...,vez|dólares|primer|ganador|dos|equivoca|número...,elconfidencial,https://www.elconfidencial.com/alma-corazon-vi...
39350,2019-10-23,"El Premio Nacional de Narrativa, por Milena Bu...",Leo con enorme alivio y regocijo las declaraci...,ser|vez|escritor|si|valor|solapa|prostitutas|b...,elperiodico,https://www.elperiodico.com/es/opinion/2019102...
39351,2019-10-24,¿Dónde ve oportunidades en los mercados? por V...,¿Dónde ve oportunidades en los mercados?\n\n¿S...,dónde|responderá|regístrese|ve|riesgos|puede|p...,expansion,https://www.expansion.com/encuentros/victor-de...


In [8]:
df.groupby(by='media')['title'].count().reset_index(name='count').sort_values('count', ascending=False)

,media,count
8,elpais,6261
1,bolsamania,4221
3,elconfidencial,4031
5,eleconomista,3756
11,lavanguardia,3282
10,expansion,3055
12,okdiario,3023
9,elperiodico,2912
0,abc,2314
14,vozpopuli,1984


In [22]:
media = 'elpais'
wmd_corpus = generate_search_corpus(df, media)

39352it [00:15, 2615.51it/s]


In [23]:
print("Retrieved {} news from {}\n".format(len(wmd_corpus), media))
print(" ".join(wmd_corpus[0][:50]) + "...")

Retrieved 6261 news from elpais

rock psicodélico pink floyd siguiendo rastro leyenda aunque naturales londres capital inglesa pink floyd conjuró leyenda aquí llegaron años 60 continuar estudios configuraron banda primero tocando pequeños locales ámbito underground después recintos londinenses emblemáticos música ruta puede comenzar sur icónica central eléctrica battersea ir ascendiendo hacia estudios abbey road grabaron...


In [5]:
w2v = Word2Vec.load('../../data/w2v_sg_d300_mc5_w5.pkl')

# Word Mover's Distance (WMD)

> *M. J. Kusner et al., From Word Embeddings To Document Distances* http://proceedings.mlr.press/v37/kusnerb15.pdf

Minimum (weighted) cumulative cost requiered to move all words from $\boldsymbol{d}_1$ to $\boldsymbol{d}_2$

$\boxed{min_{\boldsymbol{T}\geqslant{0}} \sum_{i,j=1}^{n} T_{ij} c(i,j)}$

where,

$\sum_{j=1}^{n} T_{ij}=d_i , \forall i \in \{1, ..., n\}$

$\sum_{i=1}^{n} T_{ij}=d_j , \forall j \in \{1, ..., n\}$

In [39]:
np.random.seed(2019)
rnd_index = np.random.choice(range(len(wmd_corpus)), size=2000, replace=False)

In [40]:
wmd_corpus_subsampled = [wmd_corpus[i] for i in rnd_index]

In [41]:
df_subsampled = df.loc[df['media'] == media]
df_subsampled.reset_index(drop=True, inplace=True)
df_subsampled = df_subsampled.loc[rnd_index.tolist()]
df_subsampled.groupby(by='media').count()

,date,title,text,keywords,url
media,,,,,
elpais,2000,2000,2000,2000,2000


In [42]:
num_best = 10
index = WmdSimilarity(
    wmd_corpus_subsampled,
    w2v,
    num_best=10,
    normalize_w2v_and_replace=True
)

In [43]:
%%time
#sent = 'Caída de las acciones de Telefónica, Alvarez Pallete plantea un plan de bajas voluntarias para mayores de 53 años.'
#sent = 'Negociaciones para el pacto de gobierno entre PSOE y Unidas Podemos para evitar repeticion electoral.'
#sent = 'Sigue la búsqueda de la desaparecida Blanca Fernández Ochoa, esquiadora de Cercedilla.'
#sent = 'Boris Johnson plantea un brexit duro para la salida de la unión Europea.'
sent = 'Alicante, Orihuela afectada por las inundaciones provocadas por las lluvias torrenciales del DANA.'
#sent = 'Incendios en el Amazonas queman miles de hectáreas y crean tensión entre Macrón y Bolsonaro.'
#sent = 'Iñigo Errejón se presenta a las elecciones generales con Más País en las provincias con mayor circunscripción.'
query = preprocess(sent)

sims = index[query]  # A query is simply a "look-up" in the similarity class.
print(sims)

[(1115, 0.4745356637985376), (998, 0.46729975530731205), (1602, 0.46706819794022386), (203, 0.4589042803326058), (359, 0.45765305536221107), (1632, 0.4502073050835806), (1361, 0.4487842156082014), (1315, 0.4476928429704906), (1676, 0.44752450626188167), (1156, 0.4470963067805243)]
CPU times: user 3min 14s, sys: 0 ns, total: 3min 14s
Wall time: 3min 14s


In [38]:
print('Query:')
print(sent)
print(len(sent)*"#")
for i in range(num_best):
    print("\n")
    print('sim = {:.4}'.format(sims[i][1]))
    print("Title: {} \n\nText: {}".format(df_subsampled.iloc[sims[i][0], 1],
                                          df_subsampled.iloc[sims[i][0], 2]))
    print(100*"-")

Query:
Alicante, Orihuela afectada por las inundaciones provocadas por las lluvias torrenciales del DANA.
##################################################################################################


sim = 0.4673
Title: Las lluvias torrenciales remiten en Alicante y Valencia pero arrecian en Murcia y Almería 

Text: La gota fría, que azota desde el lunes al área mediterránea, entró el miércoles por la noche en su fase más adversa, que durará hasta que acabe el viernes. Tras un rápido descenso de norte a sur por el este, la depresión aislada en niveles altos (dana), es decir, un embolsamiento de aire en capas altas conocido popularmente como gota fría— se encuentra este jueves en el mar, entre el norte de Argelia y el sureste de la Península, desde donde recibe "combustible" para generar más tormentas y hacer que sean torrenciales y duraderas, explica la Agencia Estatal de Meteorología (Aemet). Almería, Alicante y Murcia continúan esta tarde bajo aviso rojo, el máximo de los tres